In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set the path to your dataset
data_dir ='/content/drive/MyDrive/Military vehicles detection/train'

# Create a data generator
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

# Load the training data
train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

# Load the validation data
validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(train_generator.num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=10  # Adjust the number of epochs as needed
)

loss, accuracy = model.evaluate(validation_generator)
print(f'Validation loss: {loss}')
print(f'Validation accuracy: {accuracy}')

model.save('vehicle_model.h5')

Found 8384 images belonging to 10 classes.
Found 2090 images belonging to 10 classes.
Epoch 1/10
262/262 [==============================] - 2474s 9s/step - loss: 2.3143 - accuracy: 0.1252 - val_loss: 2.2401 - val_accuracy: 0.1788
Epoch 2/10
262/262 [==============================] - 547s 2s/step - loss: 2.1689 - accuracy: 0.2131 - val_loss: 2.0839 - val_accuracy: 0.2683
Epoch 3/10
262/262 [==============================] - 552s 2s/step - loss: 1.8473 - accuracy: 0.3587 - val_loss: 1.8857 - val_accuracy: 0.3327
Epoch 4/10
262/262 [==============================] - 597s 2s/step - loss: 1.2954 - accuracy: 0.5607 - val_loss: 1.8030 - val_accuracy: 0.4024
Epoch 5/10
262/262 [==============================] - 550s 2s/step - loss: 0.7164 - accuracy: 0.7644 - val_loss: 1.9443 - val_accuracy: 0.4298
Epoch 6/10
262/262 [==============================] - 569s 2s/step - loss: 0.3670 - accuracy: 0.8859 - val_loss: 2.2451 - val_accuracy: 0.4380
Epoch 7/10
262/262 [==============================] - 5

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
import cv2
import numpy as np
import tensorflow as tf

# Load the model
model = tf.keras.models.load_model('vehicle_model.h5')

# Class names mapping
class_names = {
    0: 'Anti-aircraft',
    1: 'Armored combat support vehicles',
    2: 'Armored personnel carriers',
    3: 'Infantry fighting vehicles',
    4: 'Light armored vehicles',
    5: 'Mine-protected vehicles',
    6: 'Prime movers and trucks',
    7: 'Self-propelled artillery',
    8: 'Light utility vehicles',
    9: 'Tanks'
}
print(class_names)

# Function to preprocess the image
def preprocess_image(image_path):
    img = cv2.imread(image_path)
    if img is None:
        raise FileNotFoundError(f"Image file '{image_path}' not found or cannot be opened.")
    original_img = img.copy()
    img = cv2.resize(img, (150, 150))
    img = img.astype('float32') / 255.0
    img = np.expand_dims(img, axis=0)
    return img, original_img

# Predict on a new image
image_path = '/content/drive/MyDrive/Military vehicles detection/train/tanks/tanks_0_1538.jpeg'
try:
    img, original_img = preprocess_image(image_path)
    predictions = model.predict(img)
    predicted_class = np.argmax(predictions[0])
    predicted_class_name = class_names[predicted_class]
    print(f'Predicted class: {predicted_class_name}')

    # Draw rectangle and class name on the original image
    height, width, _ = original_img.shape
    cv2.rectangle(original_img, (10, 10), (width - 10, height - 10), (0, 255, 0), 2)
    cv2.putText(original_img, f'Class: {predicted_class_name}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Display the image
    #cv2.imshow('Detected Image', original_img)
    #cv2.waitKey(0)
   # cv2.destroyAllWindows()
except FileNotFoundError as e:
    print(e)

{0: 'Anti-aircraft', 1: 'Armored combat support vehicles', 2: 'Armored personnel carriers', 3: 'Infantry fighting vehicles', 4: 'Light armored vehicles', 5: 'Mine-protected vehicles', 6: 'Prime movers and trucks', 7: 'Self-propelled artillery', 8: 'Light utility vehicles', 9: 'Tanks'}
1/1 [==============================] - 0s 126ms/step
Predicted class: Tanks
